In [42]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import logging
from pathlib import Path
import json
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, Any, Tuple, List, Union

In [43]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.ensemble import IsolationForest

In [44]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, Flatten, Bidirectional, MultiHeadAttention, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [45]:
import shap
import holidays  # Ensure holidays module is imported

Define all classes and functions from the individual files

In [46]:
class ExplainableAI:
    def __init__(self, output_dir: str):
        self.output_dir = output_dir
        self.explainers = {}
        self.shap_values = {}        
    def _reshape_input_data(self, data: np.ndarray, model_type: str) -> np.ndarray:
        """Reshape input data based on model type"""
        if model_type == 'lstm':
            return data.reshape((data.shape[0], 1, -1))
        return data
    
    def _create_model_wrapper(self, model, model_type: str):
        """Create prediction wrapper for different model types"""
        def predict_wrapper(data):
            reshaped_data = self._reshape_input_data(data, model_type)
            if model_type in ['lstm', 'autoencoder']:
                return model.predict(reshaped_data).flatten()
            return model.predict(data)
        return predict_wrapper
    def create_explainer(self, 
                        model, 
                        X_train: np.ndarray,
                        model_type: str,
                        n_samples: int = 50) -> None:
        """Initialize SHAP explainer based on model type"""
        X_background = shap.sample(X_train, n_samples)
        predict_fn = self._create_model_wrapper(model, model_type)
        
        if model_type in ['lstm', 'autoencoder']:
            self.explainers[model_type] = shap.KernelExplainer(
                predict_fn, 
                shap.kmeans(X_background, n_samples)
            )
        else:
            self.explainers[model_type] = shap.KernelExplainer(predict_fn, X_background)
    def compute_shap_values(self, 
                          X_test: np.ndarray, 
                          model_type: str,
                          n_samples: int = 50) -> np.ndarray:
        """Compute SHAP values for given model type"""
        if model_type not in self.explainers:
            raise ValueError(f"No explainer found for model type: {model_type}")
            
        X_sample = shap.sample(X_test, n_samples)
        self.shap_values[model_type] = self.explainers[model_type].shap_values(X_sample)
        return self.shap_values[model_type]
    def generate_feature_importance(self, 
                                 model_type: str, 
                                 features: List[str]) -> pd.DataFrame:
        """Generate feature importance ranking"""
        if model_type not in self.shap_values:
            raise ValueError(f"No SHAP values found for model type: {model_type}")
            
        shap_vals = self.shap_values[model_type]
        importance = np.abs(shap_vals).mean(0)
        
        return pd.DataFrame({
            'feature': features,
            'importance': importance,
            'abs_importance': np.abs(importance)
        }).sort_values('abs_importance', ascending=False)
    def plot_shap_summary(self, 
                         model_type: str, 
                         features: List[str], 
                         X_test: np.ndarray) -> None:
        """Generate and save SHAP summary plots"""
        try:
            # Bar summary
            shap.summary_plot(
                self.shap_values[model_type], 
                X_test,
                feature_names=features,
                plot_type="bar",
                show=False
            )
            plt.savefig(os.path.join(self.output_dir, f'shap_summary_bar_{model_type}.png'))
            plt.close()

            # Dot summary
            shap.summary_plot(
                self.shap_values[model_type], 
                X_test,
                feature_names=features,
                show=False
            )
            plt.savefig(os.path.join(self.output_dir, f'shap_summary_dot_{model_type}.png'))
            plt.close()
        except Exception as e:
            logging.error(f"Error in SHAP summary plotting: {str(e)}")
            plt.close()
    def plot_feature_dependence(self, 
                              model_type: str, 
                              features: List[str], 
                              X_test: np.ndarray) -> None:
        """Generate and save SHAP dependence plots for each feature"""
        try:
            for idx, feature in enumerate(features):
                safe_name = ''.join(c for c in feature if c.isalnum() or c in '_- ')
                shap.dependence_plot(
                    idx,
                    self.shap_values[model_type],
                    X_test,
                    feature_names=features,
                    show=False
                )
                plt.savefig(os.path.join(
                    self.output_dir, 
                    f'shap_dependence_{model_type}_{safe_name}.png'
                ))
                plt.close()
        except Exception as e:
            logging.error(f"Error in dependence plotting: {str(e)}")
            plt.close()

In [47]:
class ModelBuilder:
    @staticmethod
    def build_basic_lstm(input_shape, output_dim=12):  # Add output_dim parameter
        input_layer = Input(shape=input_shape)
        lstm_1 = LSTM(64, return_sequences=True)(input_layer)
        lstm_2 = LSTM(32, activation='relu')(lstm_1)
        output_layer = Dense(output_dim)(lstm_2)  # Match output dimension with target
        model = Model(inputs=input_layer, outputs=output_layer)
        return model
    @staticmethod
    def build_advanced_lstm(input_shape, output_dim=12):  # Add output_dim parameter
        model = Sequential([
            LSTM(128, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            LSTM(64, return_sequences=True),
            Dropout(0.2),
            LSTM(32, activation='relu'),
            Dense(16, activation='relu'),
            Dense(output_dim)  # Match output dimension with target
        ])
        return model
    @staticmethod
    def build_bi_lstm(input_shape, output_dim=12):  # Add output_dim parameter
        """Build Bidirectional LSTM model with correct input shape handling"""
        # Flatten the middle two dimensions if we have a 4D input
        if len(input_shape) == 3:
            input_shape = (input_shape[0], input_shape[1] * input_shape[2])
            
        model = Sequential([
            Input(shape=input_shape),  # Explicit input layer
            Bidirectional(LSTM(128, return_sequences=True)),
            Bidirectional(LSTM(64, activation='relu')),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(output_dim)
        ])
        return model
    @staticmethod
    def build_attention(input_shape, output_dim=12):  # Add output_dim parameter
        """Build Attention model with correct input shape handling"""
        # Flatten the middle two dimensions if we have a 4D input
        if len(input_shape) == 3:
            input_shape = (input_shape[0], input_shape[1] * input_shape[2])
            
        input_layer = Input(shape=input_shape)
        attention_layer = MultiHeadAttention(num_heads=4, key_dim=8)(input_layer, input_layer)
        flatten_layer = Flatten()(attention_layer)
        dense_layer = Dense(64, activation='relu')(flatten_layer)
        output_layer = Dense(output_dim)(dense_layer)  # Match output dimension with target
        return Model(inputs=input_layer, outputs=output_layer)
    @staticmethod
    def build_combined_lstm_attention(input_shape, output_dim=12):  # Add output_dim parameter
        """Build Combined LSTM-Attention model with correct input shape handling"""
        # Flatten the middle two dimensions if we have a 4D input
        if len(input_shape) == 3:
            input_shape = (input_shape[0], input_shape[1] * input_shape[2])
            
        input_layer = Input(shape=input_shape)
        lstm_layer = LSTM(64, return_sequences=True)(input_layer)
        attention_layer = MultiHeadAttention(num_heads=4, key_dim=8)(lstm_layer, lstm_layer)
        attention_normalized = LayerNormalization()(attention_layer)
        flatten_layer = Flatten()(attention_normalized)
        dense_layer = Dense(64, activation='relu')(flatten_layer)
        dropout_layer = Dropout(0.2)(dense_layer)
        output_layer = Dense(output_dim)(dropout_layer)  # Match output dimension with target
        return Model(inputs=input_layer, outputs=output_layer)
    @staticmethod
    def build_model(input_shape):
        model = Sequential()
        # ...existing code...
        model.add(Dense(24 * 12, activation='linear'))  # Adjust the output layer
        model.add(layers.Reshape((24, 12)))  # Reshape to match the expected output shape
        # ...existing code...
        return model

In [48]:
class ModelTrainer:
    def __init__(self, input_shape, output_dim=12, sequence_length=24):
        """Initialize ModelTrainer with correct input shape handling"""
        # Ensure input_shape is correctly formatted
        if len(input_shape) == 2:  # If given (timesteps, features)
            self.input_shape = (input_shape[0], input_shape[1])
        elif len(input_shape) == 3:  # If given (batch, timesteps, features)
            self.input_shape = (input_shape[1], input_shape[2])
        else:
            raise ValueError(f"Unexpected input shape: {input_shape}")
        
        self.output_dim = output_dim
        self.models = {
            'bi_lstm': ModelBuilder.build_bi_lstm(self.input_shape, self.output_dim),
            'attention': ModelBuilder.build_attention(self.input_shape, self.output_dim),
            'combined': ModelBuilder.build_combined_lstm_attention(self.input_shape, self.output_dim)
        }
        self.sequence_length = sequence_length
        self.reshape_required = True  # Enable reshaping for training/inference
    def compile_models(self, learning_rate=0.001):
        for name, model in self.models.items():
            model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    def _prepare_input_data(self, X):
        """Prepare input data by reshaping if necessary"""
        if self.reshape_required and len(X.shape) == 2:
            # Split feature dimension into timesteps & features_per_timestep
            # e.g., if X.shape[1] = 24*59, we reshape to (batch_size, 24, 59)
            if X.shape[1] % self.sequence_length != 0:
                raise ValueError("Feature dimension must be a multiple of sequence_length.")
            features_per_timestep = X.shape[1] // self.sequence_length
            X = X.reshape((X.shape[0], self.sequence_length, features_per_timestep))
        return X
    def train_model(self, model, X_train, y_train, **kwargs):
        """Train a single model with proper input data preparation"""
        X_train_prepared = self._prepare_input_data(X_train)
        
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
        ]
        
        return model.fit(
            X_train_prepared, y_train,
            validation_split=0.2,
            callbacks=callbacks,
            **kwargs
        )
    def train_all_models(self, X_train, y_train, epochs=50, batch_size=32):
        results = {}
        for name, model in self.models.items():
            logging.info(f"Training {name} model...")
            history = self.train_model(
                model, X_train, y_train,
                epochs=epochs,
                batch_size=batch_size,
                verbose=1
            )
            results[name] = history
        return results

In [49]:
class ModelEvaluator:
    def __init__(self, feature_scaler=None, target_scaler=None):
        self.feature_scaler = feature_scaler
        self.target_scaler = target_scaler
    @staticmethod
    def evaluate_model(model, X_test, y_test):
        predictions = model.predict(X_test)
        predictions = predictions.reshape(-1)
        y_test = y_test.values if hasattr(y_test, 'values') else y_test
        
        return {
            'mae': mean_absolute_error(y_test, predictions),
            'mse': mean_squared_error(y_test, predictions),
            'rmse': np.sqrt(mean_squared_error(y_test, predictions)),
            'r2': r2_score(y_test, predictions)
        }
    @staticmethod
    def cross_validate(models, X, y, n_splits=5):
        tscv = TimeSeriesSplit(n_splits=n_splits)
        cv_results = {}
        
        for name, model in models.items():
            metrics = {'mae': [], 'rmse': [], 'r2': []}
            
            for train_idx, val_idx in tscv.split(X):
                X_train_cv, X_val_cv = X[train_idx], X[val_idx]
                y_train_cv, y_val_cv = y[train_idx], y[val_idx]
                
                # Ensure proper data shape
                if len(X_train_cv.shape) == 2:
                    X_train_cv = X_train_cv.reshape((X_train_cv.shape[0], 1, X_train_cv.shape[1]))
                    X_val_cv = X_val_cv.reshape((X_val_cv.shape[0], 1, X_val_cv.shape[1]))
                
                # Train and evaluate
                early_stopping = EarlyStopping(monitor='val_loss', patience=5)
                model.fit(X_train_cv, y_train_cv, epochs=50, batch_size=32,
                         validation_split=0.2, callbacks=[early_stopping], verbose=0)
                
                # Calculate metrics
                val_metrics = ModelEvaluator.evaluate_model(model, X_val_cv, y_val_cv)
                metrics['mae'].append(val_metrics['mae'])
                metrics['rmse'].append(val_metrics['rmse'])
                metrics['r2'].append(val_metrics['r2'])
            
            # Calculate average metrics
            cv_results[name] = {
                'mae_mean': np.mean(metrics['mae']),
                'mae_std': np.std(metrics['mae']),
                'rmse_mean': np.mean(metrics['rmse']),
                'rmse_std': np.std(metrics['rmse']),
                'r2_mean': np.mean(metrics['r2']),
                'r2_std': np.std(metrics['r2'])
            }
            
        return cv_results
    def find_optimal_settings(self, model, autoencoder, features, feature_columns):
        """
        Find optimal temperature and current settings for minimum power consumption
        while maintaining comfort conditions.
        
        Returns:
        - Dictionary containing:
            - optimal_settings: DataFrame row with optimal parameters
            - min_power: Minimum predicted power consumption
            - optimal_temp: Optimal temperature
            - optimization_space: DataFrame with all evaluated combinations
        """
        # Define search spaces
        temperature_range = np.arange(18, 30, 1)
        current_range = np.arange(5, 15, 0.5)
        temp_grid, curr_grid = np.meshgrid(temperature_range, current_range)
        
        # Create parameter combinations
        settings_df = pd.DataFrame({
            'Inside Temperature (Â°C)': temp_grid.flatten(),
            'Current (A)': curr_grid.flatten()
        })
        
        # Add constant parameters
        settings_df['Voltage (V)'] = 220.0
        settings_df['Power Factor'] = 0.9
        settings_df['Outside Temperature (Â°C)'] = 25.0
        settings_df['Inside Humidity (%)'] = 50.0
        settings_df['Outside Humidity (%)'] = 50.0
        
        # Calculate derived features
        settings_df['Temperature Difference'] = (
            settings_df['Outside Temperature (Â°C)'] - 
            settings_df['Inside Temperature (Â°C)']
        )
        settings_df['Power Factor * Current'] = (
            settings_df['Power Factor'] * settings_df['Current (A)']
        )
        settings_df['Current Squared'] = settings_df['Current (A)'] ** 2
        
        # Normalize features
        if self.feature_scaler:
            normalized_data = self.feature_scaler.transform(settings_df[feature_columns])
            normalized_df = pd.DataFrame(normalized_data, columns=feature_columns)
        else:
            normalized_df = settings_df[feature_columns]
        
        # Prepare input for prediction
        model_input = normalized_df[features].values.reshape((-1, 1, len(features)))
        
        # Get predictions
        predictions = model.predict(model_input)
        if self.target_scaler:
            predictions = self.target_scaler.inverse_transform(predictions)
        
        # Get anomaly scores if autoencoder is provided
        if autoencoder is not None:
            reconstruction = autoencoder.predict(normalized_df[features].values)
            reconstruction_error = np.mean(np.power(
                normalized_df[features].values - reconstruction, 2
            ), axis=1)
            settings_df['anomaly_score'] = reconstruction_error
        
        # Add predictions to results
        settings_df['predicted_power'] = predictions.flatten()
        
        # Apply comfort constraints
        valid_mask = (
            (settings_df['Inside Temperature (Â°C)'] >= 21) & 
            (settings_df['Inside Temperature (Â°C)'] <= 25) & 
            (settings_df['Inside Humidity (%)'] >= 40) & 
            (settings_df['Inside Humidity (%)'] <= 60)
        )
        
        if autoencoder is not None:
            threshold = np.percentile(settings_df['anomaly_score'], 95)
            valid_mask &= (settings_df['anomaly_score'] < threshold)
        
        # Find optimal settings
        valid_settings = settings_df[valid_mask]
        if len(valid_settings) > 0:
            optimal_idx = valid_settings['predicted_power'].idxmin()
            optimal_settings = valid_settings.loc[optimal_idx]
            min_power = optimal_settings['predicted_power']
            optimal_temp = optimal_settings['Inside Temperature (Â°C)']
        else:
            optimal_settings = None
            min_power = None
            optimal_temp = None
        
        return {
            'optimal_settings': optimal_settings,
            'min_power': min_power,
            'optimal_temp': optimal_temp,
            'optimization_space': settings_df
        }

In [50]:
class HVACDataPreprocessor:
    """
    Comprehensive data preprocessing pipeline for HVAC system data.
    Handles missing values, feature engineering, and data validation.
    """
    
    def __init__(self, 
                 scaler_type: str = 'standard',
                 imputer_n_neighbors: int = 5,
                 country_holidays: str = 'US'):
        """
        Initialize the preprocessor with configuration parameters.
        
        Args:
            scaler_type: Type of scaler ('standard' or 'minmax')
            imputer_n_neighbors: Number of neighbors for KNN imputation
            country_holidays: Country code for holiday detection
        """
        self.scaler_type = scaler_type
        self.scaler = StandardScaler() if scaler_type == 'standard' else MinMaxScaler()
        self.imputer = KNNImputer(n_neighbors=imputer_n_neighbors)
        self.holidays = holidays.CountryHoliday(country_holidays)
        self.feature_names = None
        self.numerical_columns = None
        self._setup_logging()
        
    def _setup_logging(self):
        """Configure logging for the preprocessor"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)
    def _validate_raw_data(self, df: pd.DataFrame) -> None:
        """
        Validate the input data structure and content.
        
        Args:
            df: Input DataFrame
        Raises:
            ValueError: If data validation fails
        """
        required_columns = [
            'Date', 'on_off', 'damper', 'active_energy', 'co2_1', 'amb_humid_1',
            'active_power', 'pot_gen', 'high_pressure_1', 'high_pressure_2',
            'low_pressure_1', 'low_pressure_2', 'high_pressure_3', 'low_pressure_3',
            'outside_temp', 'outlet_temp', 'inlet_temp', 'summer_setpoint_temp',
            'winter_setpoint_temp', 'amb_temp_2'
        ]
        
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")
            
        # Check data types
        try:
            pd.to_datetime(df['Date'])
        except:
            raise ValueError("Date column cannot be parsed as datetime")
    def _handle_missing_values(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Handle missing values using appropriate strategies.
        
        Args:
            df: Input DataFrame
        Returns:
            DataFrame with handled missing values
        """
        # Fill boolean columns
        boolean_columns = ['on_off', 'damper']
        df[boolean_columns] = df[boolean_columns].fillna(0)
        
        # Handle numerical columns with KNN imputation
        numerical_columns = df.select_dtypes(include=[np.number]).columns
        df[numerical_columns] = self.imputer.fit_transform(df[numerical_columns])
        
        return df
    def _engineer_time_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Create time-based features from the Date column.
        
        Args:
            df: Input DataFrame
        Returns:
            DataFrame with additional time-based features
        """
        df['datetime'] = pd.to_datetime(df['Date'])
        
        # Extract basic time components
        df['hour'] = df['datetime'].dt.hour
        df['day_of_week'] = df['datetime'].dt.dayofweek
        df['month'] = df['datetime'].dt.month
        df['is_weekend'] = df['datetime'].dt.dayofweek.isin([5, 6]).astype(int)
        
        # Create cyclical time features
        df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
        df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
        df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
        
        # Add holiday indicator
        df['is_holiday'] = df['datetime'].apply(lambda x: x in self.holidays).astype(int)
        
        return df
    def _engineer_hvac_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Create HVAC-specific engineered features.
        
        Args:
            df: Input DataFrame
        Returns:
            DataFrame with additional HVAC-specific features
        """
        # Temperature differences
        df['temp_difference_in_out'] = df['outlet_temp'] - df['inlet_temp']
        df['temp_difference_ambient'] = df['outside_temp'] - df['inlet_temp']
        
        # Pressure ratios and differences
        df['high_pressure_avg'] = df[['high_pressure_1', 'high_pressure_2', 'high_pressure_3']].mean(axis=1)
        df['low_pressure_avg'] = df[['low_pressure_1', 'low_pressure_2', 'low_pressure_3']].mean(axis=1)
        df['pressure_ratio'] = df['high_pressure_avg'] / df['low_pressure_avg']
        
        # System efficiency indicators
        df['power_per_temp_diff'] = df['active_power'] / (df['temp_difference_in_out'] + 1e-6)
        df['energy_efficiency'] = df['active_energy'] / (df['active_power'] + 1e-6)
        
        # Comfort indicators
        df['temp_setpoint_diff'] = np.where(
            df['month'].isin([6, 7, 8]),  # Summer months
            df['inlet_temp'] - df['summer_setpoint_temp'],
            df['inlet_temp'] - df['winter_setpoint_temp']
        )
        
        return df
    def _create_rolling_features(self, df: pd.DataFrame, windows: List[int] = [3, 6, 12]) -> pd.DataFrame:
        """
        Create rolling window features for key metrics.
        
        Args:
            df: Input DataFrame
            windows: List of window sizes (in hours) for rolling calculations
        Returns:
            DataFrame with additional rolling features
        """
        key_metrics = ['active_power', 'inlet_temp', 'co2_1', 'amb_humid_1']
        
        for window in windows:
            for metric in key_metrics:
                # Rolling mean
                df[f'{metric}_rolling_mean_{window}h'] = (
                    df[metric].rolling(window=window * 12, min_periods=1).mean()
                )
                # Rolling std
                df[f'{metric}_rolling_std_{window}h'] = (
                    df[metric].rolling(window=window * 12, min_periods=1).std()
                )
        
        return df
    def _prepare_target_variable(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
        """
        Prepare the target variable (active_power) for modeling.
        
        Args:
            df: Input DataFrame
        Returns:
            Tuple of (features DataFrame, target Series)
        """
        target = df['active_power']
        features = df.drop(['active_power', 'datetime', 'Date'], axis=1)
        
        return features, target
    def _scale_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Scale numerical features using the configured scaler.
        
        Args:
            df: Input DataFrame
        Returns:
            DataFrame with scaled features
        """
        numerical_columns = df.select_dtypes(include=[np.number]).columns
        df[numerical_columns] = self.scaler.fit_transform(df[numerical_columns])
        return df
    def preprocess(self, 
                  df: pd.DataFrame, 
                  training: bool = True) -> Tuple[pd.DataFrame, pd.Series]:
        """
        Execute the complete preprocessing pipeline.
        
        Args:
            df: Input DataFrame
            training: Whether this is for training data (True) or inference (False)
        Returns:
            Tuple of (preprocessed features, target variable)
        """
        self.logger.info("Starting preprocessing pipeline...")
        
        # Validate raw data
        self._validate_raw_data(df)
        
        # Handle missing values
        df = self._handle_missing_values(df)
        
        # Engineer features
        df = self._engineer_time_features(df)
        df = self._engineer_hvac_features(df)
        df = self._create_rolling_features(df)
        
        # Prepare features and target
        features, target = self._prepare_target_variable(df)
        
        # Scale features
        features = self._scale_features(features)
        
        if training:
            self.feature_names = features.columns.tolist()
            self.numerical_columns = features.select_dtypes(include=[np.number]).columns.tolist()
        
        self.logger.info("Preprocessing pipeline completed successfully.")
        return features, target
    def get_feature_names(self) -> List[str]:
        """Return the list of feature names after preprocessing"""
        if self.feature_names is None:
            raise ValueError("Preprocessor hasn't been fitted with training data yet")
        return self.feature_names

In [51]:
class DataValidator:
    """
    Validates data quality and generates reports on data issues.
    """
    
    @staticmethod
    def check_data_quality(df: pd.DataFrame) -> Dict[str, Union[float, List[str]]]:
        """
        Perform comprehensive data quality checks.
        
        Args:
            df: Input DataFrame
        Returns:
            Dictionary containing quality metrics and issues
        """
        quality_report = {
            'missing_values': {},
            'outliers': {},
            'inconsistencies': [],
            'data_coverage': {}
        }
        
        # Check missing values
        missing_vals = df.isnull().sum()
        quality_report['missing_values'] = {
            col: count for col, count in missing_vals.items() if count > 0
        }
        
        # Check value ranges
        for col in df.select_dtypes(include=[np.number]).columns:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            outliers = df[
                (df[col] < (q1 - 1.5 * iqr)) | 
                (df[col] > (q3 + 1.5 * iqr))
            ]
            if len(outliers) > 0:
                quality_report['outliers'][col] = len(outliers)
        
        # Check data consistency
        if 'datetime' in df.columns:
            time_gaps = df['datetime'].diff().dt.total_seconds() / 60
            irregular_intervals = time_gaps[time_gaps != 5].index
            if len(irregular_intervals) > 0:
                quality_report['inconsistencies'].append(
                    f"Irregular time intervals found at {len(irregular_intervals)} points"
                )
        
        # Check data coverage
        if 'datetime' in df.columns:
            date_range = df['datetime'].max() - df['datetime'].min()
            expected_records = (date_range.total_seconds() / 300) + 1  # 5-minute intervals
            coverage = len(df) / expected_records * 100
            quality_report['data_coverage'] = {
                'start_date': df['datetime'].min(),
                'end_date': df['datetime'].max(),
                'coverage_percentage': coverage
            }
        
        return quality_report

In [52]:
class AnomalyDetector:
    def __init__(self, input_dim, method='autoencoder'):
        self.input_dim = input_dim
        self.method = method
        self.model = self._build_model()
        self.threshold = None
        self.scaler = StandardScaler()
        
    def _build_model(self):
        if self.method == 'autoencoder':
            return self._build_basic_autoencoder()
        elif self.method == 'complex_autoencoder':
            return self._build_complex_autoencoder()
        elif self.method == 'isolation_forest':
            return IsolationForest(contamination=0.1, random_state=42)
        else:
            raise ValueError(f"Unknown method: {self.method}")
    def _build_basic_autoencoder(self):
        input_layer = Input(shape=(self.input_dim,))
        # Encoder
        encoder = Dense(32, activation="relu")(input_layer)
        encoder = Dense(16, activation="relu")(encoder)
        encoder = Dropout(0.2)(encoder)
        # Decoder
        decoder = Dense(32, activation="relu")(encoder)
        decoder = Dense(self.input_dim, activation="linear")(decoder)
        
        model = Model(inputs=input_layer, outputs=decoder)
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        return model
    def _build_complex_autoencoder(self):
        input_layer = Input(shape=(self.input_dim,))
        # Enhanced encoder
        encoded = Dense(64, activation='relu')(input_layer)
        encoded = LayerNormalization()(encoded)
        encoded = Dense(32, activation='relu')(encoded)
        encoded = Dropout(0.2)(encoded)
        encoded = Dense(16, activation='relu')(encoded)
        encoded = LayerNormalization()(encoded)
        
        # Enhanced decoder
        decoded = Dense(32, activation='relu')(encoded)
        decoded = LayerNormalization()(decoded)
        decoded = Dense(64, activation='relu')(decoded)
        decoded = Dropout(0.2)(decoded)
        decoded = LayerNormalization()(decoded)
        decoded = Dense(self.input_dim, activation='linear')(decoded)
        
        model = Model(inputs=input_layer, outputs=decoded)
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        return model
    def fit(self, X, epochs=50, batch_size=32):
        X_scaled = self.scaler.fit_transform(X)
        
        if isinstance(self.model, Model):  # Autoencoder models
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=5,
                restore_best_weights=True
            )
            
            history = self.model.fit(
                X_scaled, X_scaled,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=0.2,
                callbacks=[early_stopping],
                verbose=0
            )
            
            # Set threshold based on reconstruction error
            predictions = self.model.predict(X_scaled)
            reconstruction_errors = np.mean(np.power(X_scaled - predictions, 2), axis=1)
            self.threshold = np.percentile(reconstruction_errors, 95)
            
            return history
        else:  # Isolation Forest
            self.model.fit(X_scaled)
            return None
    def detect_anomalies(self, X, threshold=None):
        X_scaled = self.scaler.transform(X)
        
        if isinstance(self.model, Model):  # Autoencoder models
            predictions = self.model.predict(X_scaled)
            reconstruction_errors = np.mean(np.power(X_scaled - predictions, 2), axis=1)
            threshold = threshold or self.threshold
            return {
                'anomalies': reconstruction_errors > threshold,
                'scores': reconstruction_errors,
                'threshold': threshold
            }
        else:  # Isolation Forest
            predictions = self.model.predict(X_scaled)
            return {
                'anomalies': predictions == -1,
                'scores': self.model.score_samples(X_scaled),
                'threshold': None
            }
    def get_reconstruction(self, X):
        """Get reconstructed data for autoencoder models"""
        if not isinstance(self.model, Model):
            raise ValueError("Reconstruction only available for autoencoder models")
        
        X_scaled = self.scaler.transform(X)
        reconstructed = self.model.predict(X_scaled)
        return self.scaler.inverse_transform(reconstructed)

In [53]:
class AnomalyDetectorEnsemble:
    def __init__(self, input_dim):
        self.detectors = {
            'basic_autoencoder': AnomalyDetector(input_dim, 'autoencoder'),
            'complex_autoencoder': AnomalyDetector(input_dim, 'complex_autoencoder'),
            'isolation_forest': AnomalyDetector(input_dim, 'isolation_forest')
        }
        
    def fit(self, X, epochs=50, batch_size=32):
        results = {}
        for name, detector in self.detectors.items():
            results[name] = detector.fit(X, epochs, batch_size)
        return results
    
    def detect_anomalies(self, X):
        results = {}
        for name, detector in self.detectors.items():
            results[name] = detector.detect_anomalies(X)
        return results
    
    def get_ensemble_predictions(self, X, voting='majority'):
        all_predictions = self.detect_anomalies(X)
        
        if voting == 'majority':
            # Stack predictions from all detectors
            predictions = np.column_stack([
                pred['anomalies'] for pred in all_predictions.values()
            ])
            # Return True if majority of detectors flag as anomaly
            return np.mean(predictions, axis=1) > 0.5
        
        return all_predictions

In [54]:
class HVACAnalysisPipeline:
    """
    Main pipeline class that coordinates data processing, model training,
    and result generation for HVAC system analysis.
    """
    
    def __init__(self, config: Dict[str, Any]):
        """
        Initialize the pipeline with configuration parameters.
        
        Args:
            config: Dictionary containing configuration parameters
        """
        self.config = config
        self.setup_directories()
        self.setup_logging()
        
    def setup_directories(self) -> None:
        """Create necessary directories for outputs"""
        # Create timestamp-based output directory
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.output_dir = Path(self.config['output_base_dir']) / f"analysis_{timestamp}"
        
        # Create subdirectories
        self.subdirs = {
            'models': self.output_dir / 'models',
            'plots': self.output_dir / 'plots',
            'results': self.output_dir / 'results',
            'anomalies': self.output_dir / 'anomalies',
            'shap': self.output_dir / 'shap'
        }
        
        for dir_path in self.subdirs.values():
            dir_path.mkdir(parents=True, exist_ok=True)
            
    def setup_logging(self) -> None:
        """Configure logging for the pipeline"""
        log_file = self.output_dir / 'pipeline.log'
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
        
    def load_data(self) -> pd.DataFrame:
        """
        Load data from CSV file and perform initial validation.
        
        Returns:
            DataFrame containing raw data
        """
        self.logger.info(f"Loading data from {self.config['data_path']}")
        try:
            df = pd.read_csv(self.config['data_path'])
            self.logger.info(f"Loaded {len(df)} records")
            return df
        except Exception as e:
            self.logger.error(f"Error loading data: {str(e)}")
            raise
            
    def save_results(self, results: Dict[str, Any], filename: str) -> None:
        """Save results to JSON file"""
        file_path = self.subdirs['results'] / filename
        with open(file_path, 'w') as f:
            json.dump(results, f, indent=4, default=str)
            
    def plot_model_performance(self, results: Dict[str, Any]) -> None:
        """Generate and save model performance plots"""
        # Training history plot
        plt.figure(figsize=(12, 6))
        for model_name, history in results['training_history'].items():
            plt.plot(history.history['loss'], label=f'{model_name}_train')
            plt.plot(history.history['val_loss'], label=f'{model_name}_val')
        plt.title('Model Training History')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(self.subdirs['plots'] / 'training_history.png')
        plt.close()
        
        # Model comparison plot
        plt.figure(figsize=(10, 6))
        metrics = pd.DataFrame(results['evaluation_results']).T
        metrics['r2'].plot(kind='bar')
        plt.title('Model RÂ² Scores')
        plt.tight_layout()
        plt.savefig(self.subdirs['plots'] / 'model_comparison.png')
        plt.close()
        
    def run_pipeline(self) -> Dict[str, Any]:
        """
        Execute the complete analysis pipeline.
        
        Returns:
            Dictionary containing all results and metrics
        """
        try:
            # 1. Load and preprocess data
            self.logger.info("Starting data preprocessing")
            raw_data = self.load_data()
            
            # Validate data quality
            quality_report = DataValidator.check_data_quality(raw_data)
            self.save_results(quality_report, 'data_quality_report.json')
            
            # Process data
            X_processed, y = process_hvac_data(self.config['data_path'])
            
            # 2. Split data
            split_idx = int(len(X_processed) * 0.8)
            X_train, X_test = X_processed[:split_idx], X_processed[split_idx:]
            y_train, y_test = y[:split_idx], y[split_idx:]
            
            # Preprocess data
            X_train, y_train = preprocess_data(X_train, y_train)
            
            # 3. Train regression models
            self.logger.info("Training regression models")
            trainer = ModelTrainer(
                input_shape=(X_train.shape[1], X_train.shape[2]),
                output_dim=y_train.shape[1]  # Add output dimension
            )
            trainer.compile_models()
            training_results = trainer.train_all_models(X_train, y_train)
            
            # 4. Evaluate models
            self.logger.info("Evaluating models")
            evaluator = ModelEvaluator()
            evaluation_results = {}
            for name, model in trainer.models.items():
                evaluation_results[name] = evaluator.evaluate_model(model, X_test, y_test)
            
            # 5. Detect anomalies
            self.logger.info("Performing anomaly detection")
            anomaly_detector = AnomalyDetectorEnsemble(X_train.shape[2])
            anomaly_detector.fit(X_train)
            anomaly_results = anomaly_detector.detect_anomalies(X_test)
            
            # 6. Generate SHAP explanations
            self.logger.info("Generating model explanations")
            explainer = ExplainableAI(str(self.subdirs['shap']))
            shap_results = analyze_model_explanability(
                trainer.models,
                X_train,
                X_test,
                self.config['feature_names']
            )
            
            # 7. Compile and save results
            results = {
                'training_history': training_results,
                'evaluation_results': evaluation_results,
                'anomaly_results': anomaly_results,
                'shap_results': shap_results
            }
            
            self.save_results(results, 'analysis_results.json')
            
            # 8. Generate plots
            self.plot_model_performance(results)
            
            # 9. Generate recommendations
            recommendations = self.generate_recommendations(results)
            self.save_results(recommendations, 'recommendations.json')
            
            self.logger.info("Pipeline completed successfully")
            return results
            
        except Exception as e:
            self.logger.error(f"Pipeline failed: {str(e)}")
            raise
            
    def generate_recommendations(self, results: Dict[str, Any]) -> Dict[str, Any]:
        """
        Generate actionable recommendations based on analysis results.
        
        Args:
            results: Dictionary containing analysis results
        Returns:
            Dictionary containing recommendations
        """
        recommendations = {
            'model_selection': {},
            'anomaly_detection': {},
            'system_optimization': {}
        }
        
        # Model selection recommendations
        best_model = max(
            results['evaluation_results'].items(),
            key=lambda x: x[1]['r2']
        )[0]
        
        recommendations['model_selection'] = {
            'best_model': best_model,
            'performance_metrics': results['evaluation_results'][best_model],
            'reason': f"Selected based on highest RÂ² score of {results['evaluation_results'][best_model]['r2']:.3f}"
        }
        
        # Anomaly detection recommendations
        anomaly_count = sum(results['anomaly_results']['ensemble_predictions'])
        recommendations['anomaly_detection'] = {
            'anomaly_count': int(anomaly_count),
            'anomaly_percentage': float(anomaly_count / len(results['anomaly_results']['ensemble_predictions']) * 100),
            'suggestion': "Investigate system behavior during identified anomaly periods"
        }
        
        # System optimization recommendations
        if 'optimization_results' in results:
            opt = results['optimization_results']
            recommendations['system_optimization'] = {
                'optimal_temperature': float(opt['optimal_temp']),
                'estimated_savings': float(opt['min_power']),
                'implementation_steps': [
                    "Gradually adjust setpoints to optimal values",
                    "Monitor system performance during transition",
                    "Validate energy savings after implementation"
                ]
            }
            
        return recommendations

In [55]:
def preprocess_data(X, y, sequence_length=24, forecast_horizon=12):
    """
    Preprocess time series data into sequences for training.
    
    Args:
        X: Input features array (2D: samples, features)
        y: Target values array (1D)
        sequence_length: Length of input sequences
        forecast_horizon: Number of future steps to predict
    
    Returns:
        Tuple of (X sequences, y sequences)
    """
    # Ensure y is 1D
    if isinstance(y, pd.Series):
        y = y.values
    if len(y.shape) > 1:
        y = y.ravel()
    
    # Calculate valid number of sequences
    num_samples = len(y) - sequence_length - forecast_horizon + 1
    
    if num_samples <= 0:
        raise ValueError("Not enough samples to create sequences")
    
    # Initialize arrays with correct shapes
    num_features = X.shape[-1]
    X_sequences = np.zeros((num_samples, sequence_length, num_features))
    y_sequences = np.zeros((num_samples, forecast_horizon))
    
    # Create sequences without reshaping input
    for i in range(num_samples):
        X_sequences[i] = X[i:i + sequence_length]
        y_sequences[i] = y[i + sequence_length:i + sequence_length + forecast_horizon]
    
    return X_sequences, y_sequences

In [56]:
def analyze_model_explanability(
    models: Dict[str, BaseEstimator],
    X_train: np.ndarray,
    X_test: np.ndarray,
    features: List[str],
    output_dir: str
) -> Dict:
    """
    Comprehensive analysis of model explanability for both supervised and unsupervised models.
    
    Returns:
    - Dictionary containing feature importance and SHAP values for each model
    """
    explainer = ExplainableAI(output_dir)
    results = {}
    
    for model_name, model in models.items():
        try:
            # Create explainer and compute SHAP values
            explainer.create_explainer(model, X_train, model_name)
            shap_values = explainer.compute_shap_values(X_test, model_name)
            
            # Generate importance analysis
            importance_df = explainer.generate_feature_importance(model_name, features)
            
            # Create visualizations
            explainer.plot_shap_summary(model_name, features, X_test)
            explainer.plot_feature_dependence(model_name, features, X_test)
            
            results[model_name] = {
                'shap_values': shap_values,
                'feature_importance': importance_df
            }
            
        except Exception as e:
            logging.error(f"Error analyzing model {model_name}: {str(e)}")
    

In [57]:
def prepare_sequence_data(features: pd.DataFrame, 
                         target: pd.Series,
                         sequence_length: int = 24,
                         forecast_horizon: int = 12) -> Tuple[np.ndarray, np.ndarray]:
    """
    Prepare sequential data for time series models.
    
    Args:
        features: Preprocessed feature DataFrame
        target: Target variable Series
        sequence_length: Number of time steps in each sequence
        forecast_horizon: Number of steps to forecast
    Returns:
        Tuple of (X sequences, y sequences)
    """
    X, y = [], []
    
    for i in range(len(features) - sequence_length - forecast_horizon + 1):
        X.append(features.iloc[i:(i + sequence_length)].values)
        y.append(target.iloc[i + sequence_length:i + sequence_length + forecast_horizon].values)
    
    return np.array(X), np.array(y)

In [58]:
def perform_hyperparameter_tuning(X_train, y_train, input_shape):
    """Single implementation of hyperparameter tuning"""
    
    class CustomLSTMRegressor(BaseEstimator, RegressorMixin):
        def __init__(self, neurons=64, learning_rate=0.001, epochs=50, batch_size=32):
            self.neurons = neurons
            self.learning_rate = learning_rate
            self.epochs = epochs
            self.batch_size = batch_size
            self.model = None
        def fit(self, X, y):
            self.model = ModelBuilder.build_basic_lstm(input_shape)
            self.model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='mse')
            self.model.fit(
                X, y,
                epochs=self.epochs,
                batch_size=self.batch_size,
                validation_split=0.2,
                callbacks=[EarlyStopping(monitor='val_loss', patience=5)],
                verbose=0
            )
            return self
        def predict(self, X):
            return self.model.predict(X).reshape(-1)
    param_grid = {
        'neurons': [32, 64, 128],
        'learning_rate': [0.01, 0.001, 0.0001],
        'batch_size': [16, 32, 64]
    }
    grid = GridSearchCV(
        estimator=CustomLSTMRegressor(),
        param_grid=param_grid,
        cv=TimeSeriesSplit(n_splits=3),
        scoring='neg_mean_squared_error',
        n_jobs=1
    )
    grid_result = grid.fit(X_train, y_train)
    return grid_result.best_params_, grid_result.best_score_

In [59]:
def analyze_anomalies(X_train, X_test):
    """
    Analyze anomalies using multiple methods and compare results.
    
    Returns:
    - Dictionary containing results from all methods and ensemble predictions
    """
    ensemble = AnomalyDetectorEnsemble(X_train.shape[1])
    
    # Train all detectors
    training_results = ensemble.fit(X_train)
    
    # Get predictions from all methods
    predictions = ensemble.detect_anomalies(X_test)
    
    # Get ensemble predictions
    ensemble_predictions = ensemble.get_ensemble_predictions(X_test)
    
    return {
        'individual_results': predictions,
        'ensemble_predictions': ensemble_predictions,
        'training_results': training_results
    }

In [60]:
def process_hvac_data(data_path: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Process HVAC data from raw file to model-ready format.
    
    Args:
        data_path: Path to raw data file
    Returns:
        Tuple of (processed features, target variable)
    """
    # Read data
    df = pd.read_csv(data_path, parse_dates=['Date'])
    
    # Initialize preprocessor
    preprocessor = HVACDataPreprocessor(
        scaler_type='standard',
        imputer_n_neighbors=5
    )
    
    # Validate data quality
    quality_report = DataValidator.check_data_quality(df)
    logging.info(f"Data quality report: {quality_report}")
    
    # Preprocess data
    features, target = preprocessor.preprocess(df, training=True)
    
    # Convert target to 1D array if needed
    if isinstance(target, pd.Series):
        target = target.values
    if len(target.shape) > 1:
        target = target.ravel()
    
    # Convert features to numpy array if it's a DataFrame
    if isinstance(features, pd.DataFrame):
        features = features.values
        
    return features, target

In [61]:
def train_and_evaluate_models(X_train, X_test, y_train, y_test, features, feature_scaler, target_scaler):
    # Initialize evaluator
    evaluator = ModelEvaluator(feature_scaler, target_scaler)
    
    # Train models
    trainer = ModelTrainer((1, len(features)))
    trainer.compile_models()
    training_results = trainer.train_all_models(X_train, y_train)
    
    # Evaluate models
    evaluation_results = {}
    for name, model in trainer.models.items():
        metrics = evaluator.evaluate_model(model, X_test, y_test)
        evaluation_results[name] = metrics
    
    # Find optimal settings for best model
    best_model_name = max(evaluation_results, key=lambda k: evaluation_results[k]['r2'])
    best_model = trainer.models[best_model_name]
    
    # Build autoencoder for anomaly detection
    autoencoder = ModelBuilder.build_autoencoder(len(features))
    autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, verbose=0)
    
    # Get optimal settings
    optimization_results = evaluator.find_optimal_settings(
        best_model, 
        autoencoder,
        features,
        feature_columns
    )
    
    return {
        'models': trainer.models,
        'evaluation_results': evaluation_results,
        'optimization_results': optimization_results,
        'best_model_name': best_model_name
    }